# Create and Run chat flows using Promptflow Python SDK

### Overview

Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality.

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.

#### 1. Set up Promptflow client with Credential and configuration

#### 2. Create a new chat flow by providing the flow name and description.

#### 3. Run Basic Promptflow with questions to compare models

#### 4. Run Context Added Promptflow with the outdoor questions

#### 5. Use serverless endpoint to run the Promptflow with context

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [42]:
%load_ext autoreload
%autoreload 2

import os, sys
lab_prep_dir = os.getcwd().split("slm-innovator-lab")[0] + "slm-innovator-lab/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Kernel: python31014jvsc74a57bd02139c70ac98f3202d028164a545621647e07f47fd6f5d8ac55cf952bf7c15ed1


In [43]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError
load_dotenv()
with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

3d4d3dd0-79d4-40cf-a94e-b4154812c6ca
slm-innovator-rg
slm-pjt1


In [44]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration

-   Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name


In [45]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: config.json


Connected to Azure AI Studio Workspace: slm-pjt1
Workspace Location: eastus
Workspace ID: /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


## 2. Create a new chat flow by providing the flow name and description.

-   Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI.

> ✨ **_important_** <br>
> Grant the Storage File Data Privileged Contributor, Storage Blob Data Contributor at the storage of AI studiorole to user, group, service principle and managed Identity which you are trying to access, you can execute the code below.


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [46]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_phi35_connection_name": "phi35-con",
	"your_gpt4o_connection_name": "ai-aistudiohub1553378079542_aoai"
}

rendered_content = template.render(variables)
Path('./chat/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat/flow.dag.yaml').read_text()) 

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
environment:
  python_requirements_txt: requirements.txt
inputs:
  question:
    type: string
    is_chat_input: true
    default: What is the capital of France?
outputs:
  phi35_answer:
    type: string
    reference: ${phi35.output}
    is_chat_output: false
  gpt4o_answer:
    type: string
    reference: ${gpt4o.output}
    is_chat_output: true
nodes:
- name: phi35
  type: python
  source:
    type: code
    path: phi35_finetuned.py
  inputs:
    connection: phi35-con
    input_data: ${inputs.question}
- name: gpt4o
  type: llm
  source:
    type: code
    path: chat.jinja2
  inputs:
    deployment_name: gpt-4o
    temperature: 0.7
    top_p: 1
    max_tokens: 256
    question: ${inputs.question}
  connection: ai-aistudiohub1553378079542_aoai
  api: chat
  module: promptflow.tools.aoai
  use_variants: false


In [47]:
#Grant the right accessibility to create the flow 
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")

Flow created successfully:
{
    "name": "588771f3-d6b6-403a-a919-c19678dac8e9",
    "type": "chat",
    "description": "fine-tuned model comparison flow",
    "path": "Users/hyokeunchoi/promptflow/chat-11-13-2024-11-34-47/flow.dag.yaml",
    "code": "azureml://locations/eastus/workspaces/884cbbaf-4d30-486f-a017-f9265971b9e1/flows/588771f3-d6b6-403a-a919-c19678dac8e9",
    "display_name": "comparison flow created from python sdk",
    "owner": {
        "user_object_id": "b1483c7c-8db1-40cb-8e36-7a69b6c50164",
        "user_tenant_id": "16b3c013-d300-468d-ac64-7eda0820b6d3",
        "user_name": "Hyo Choi"
    },
    "is_archived": false,
    "created_date": "2024-11-13 11:34:50.504192+00:00",
    "flow_portal_url": "https://ai.azure.com/projectflows/588771f3-d6b6-403a-a919-c19678dac8e9/884cbbaf-4d30-486f-a017-f9265971b9e1/details/Flow?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1

## 3. Run Basic Promptflow with questions to compare models

-   Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models


In [48]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


Uploading chat (0.01 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5701/5701 [00:00<00:00, 86676.89it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_variant_0_20241113_113457_547377/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [49]:
monitor_status(pf_azure_client, run_result)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:01,  1.34step/s]

Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:11<00:06,  6.61s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:06<00:00, 22.03s/step]

Promptflow Running Completed


In [50]:
detail = pf_azure_client.get_details(run_result)

detail

,inputs.question,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,
0,what is the center of Seoul?,0,The center of Seoul is often considered to be...,The center of Seoul is generally considered to...
1,What is the capital of France?,1,The capital of France is Paris.,The capital of France is Paris.
2,Tell me about the brief history of Microsoft,2,Microsoft was founded in 1975 by Bill Gates a...,Microsoft was founded by Bill Gates and Paul A...
3,Who wrote Romeo and Juliet?,3,"William Shakespeare wrote ""Romeo and Juliet.""",William Shakespeare wrote Romeo and Juliet.
4,What does HTML stand for?,4,HTML stands for HyperText Markup Language. It...,HTML stands for HyperText Markup Language.
5,Who painted the Mona Lisa?,5,Leonardo da Vinci painted the Mona Lisa.,Leonardo da Vinci painted the Mona Lisa.
6,What is the largest planet in our solar system?,6,Jupiter is the largest planet in our solar sy...,Jupiter is the largest planet in our solar sys...
7,Who won the Nobel Prize in Literature in 2024?,7,"As of my knowledge cutoff in March 2023, I ca...","I'm sorry, but I don't have information about ..."


## 4. Run Context Added Promptflow with the outdoor questions

-   Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-context.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_phi35_connection_name": "phi35-con",
	"your_gpt4o_connection_name": "ai-aistudiohub1553378079542_aoai"
}

rendered_content = template.render(variables)
Path('./chat-context/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-context/flow.dag.yaml').read_text()) 

In [ ]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_result_with_context)

In [ ]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

## 5. Use serverless endpoint to run the Promptflow with context

-   Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.


### deploy your serverless endpoint

-   go to the Azure AI studio > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint
    <br>
    ![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
    <br>
    <br>
-   once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
    ![copy connection](./images/copy_connection.jpg)
    <br>
    <br>
-   go to Settings in Azure AI studio > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
    ![create new serverless connection](./images/create_new_serverless_connection.jpg)


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-serverless.flow.dag.yaml')

# Define the variables for the template with your connection names for chat serverless 
variables = {
	"your_phi35_serverless_connection_name": "replace with your connection name",
	"your_gpt4o_connection_name": "replace with your connection name"
}

rendered_content = template.render(variables)
Path('./chat-serverless/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-serverless/flow.dag.yaml').read_text()) 

In [ ]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_serverless_result)

In [ ]:
detail = pf_azure_client.get_details(run_serverless_result)

detail